Convert Axis oriented dataset to OBB using SAM

In [8]:
from pathlib import Path
from ultralytics import SAM
from ultralytics.utils.sam_extractor import DatasetOBBExtractor

final_dir = Path("/work3/dimda/ultralytics_dotav2/examples/datasets/xView-patches-ship-sam")
model = SAM("sam_b.pt")
model.info()
debug = False
dataset_dir = final_dir
sam_ext = "sam-annos"
output_dir = final_dir / sam_ext
dat = DatasetOBBExtractor(
    model=model,
    dataset_dir=str(dataset_dir),
    output_dir=str(output_dir),
    default_class="ship",  # keep_only_classes=[0],
    debug=debug,
)
dat.get_dataset_info()
list(dat.process(idxs=None))
import subprocess

subprocess.run(["mv", str(final_dir / "labels"), str(final_dir / "labels-original")])
subprocess.run(["mv", str(output_dir / "labels"), str(final_dir / "labels")])
subprocess.run(["rmdir", str(output_dir / sam_ext)])

ModuleNotFoundError: No module named 'ultralytics'

Extract patches from the OBB dataset

In [1]:
# Extract patches dataset from original OBB dataset to a new directory
from ultralytics.utils.sam_extractor import DatasetOBBExtractor
from pathlib import Path
final_dir = Path("/work3/dimda/ultralytics_dotav2/examples/datasets/xView-patches-ship-sam")
dat = DatasetOBBExtractor(model=None, dataset_dir=final_dir, output_dir=None, default_class=None, debug=False)
dat.get_dataset_info()
patches = dat.extract_patches(idxs=None, output_dir=final_dir+"-crops")

ModuleNotFoundError: No module named 'ultralytics'

In [2]:
"""
This script prepares a dataset in the Hugging Face image classification format.

The Hugging Face image classification format expects the dataset to be structured as follows:

base_dir/
|__ class_0/
|   |__ img0.png
|   |__ img1.png
|   |__ ...
|
|__ class_1/
|   |__ img0.png
|   |__ img1.png
|   |__ ...
|
|__ ...

Each subdirectory represents a class and contains the images for that class.

"""

import os
import shutil

# Discover all files in the folder

base_dir = "/work1/dimda/xView-patches-ship-sam-crops" #"/work1/dimda/xView-patches-ship-sam-crops"
files = os.listdir(base_dir)

# Extract class number from files
for file in files:
    class_num = file.split("_")[2].split(".")[0]
    class_dir = f"{base_dir}/class_{class_num}"
    if not os.path.exists(f"{base_dir}/class_{class_num}"):
        os.makedirs(class_dir)
    shutil.move(f"{base_dir}/{file}", f"{class_dir}/{file}")

In [14]:
# Train classifier

! export SHARDED= CUDA_VISIBLE_DEVICES=0 HF_DATASETS_CACHE="/work1/dimda/cache/huggingface" \
WANDB_CACHE_DIR="/work1/dimda/cache/wandb" \
WANDB_MODE=online \
&& python3 train_classifier.py

2024-03-06 16:25:30,768 - INFO - Starting experiment named 2024-03-06 16:25:27 with project name fine_ship_classifier
wandb: Currently logged in as: dimidagd (dimidagd_sl). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /work3/dimda/ultralytics_dotav2/examples/wandb/run-20240306_162533-d62g3pbj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run 2024-03-06 16:25:27
wandb: ⭐️ View project at https://wandb.ai/dimidagd_sl/fine_ship_classifier
wandb: 🚀 View run at https://wandb.ai/dimidagd_sl/fine_ship_classifier/runs/d62g3pbj
2024-03-06 16:25:50,930 - INFO - Loading dataset
2024-03-06 16:25:50,983 - INFO - Defining label2idx and vice versa
config.json: 100%|█████████████████████████████| 502/502 [00:00<00:00, 1.03MB/s]
2024-03-06 16:25:51,810 - INFO - Loading model google/vit-base-patch32-224